# Thickness
* This Notebook is used to calculate several thickness-related propeties from the WellCad data
Main parts include:
- A cumulative thickness per geochem group function/algorithm
- XXXXXXXXXXXXXXXXXXXXXXX

## Importing Libraries and Data

In [27]:
# Importing libraries
import pandas as pd
import re

# Reading .csv
# data = pd.read_csv(r'Data/CSV/1-AV-001-PR.csv', sep=';', decimal=',') ## Dataset1
data = pd.read_csv(r'Data/CSV/1-CS-002-PR.csv', sep=';', decimal=',') ##Dataset2
data.head()

,Well,Unit,Facies,Facies Thickness,Flow thickness,Core Proportion
0,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),3.65,NaN,NaN
1,1-CS-002-PR,Paranapanema,Simple lava (basic | L. crust),1.12,NaN,NaN
2,1-CS-002-PR,Paranapanema,Siliciclastics,1.52,NaN,NaN
3,1-CS-002-PR,Paranapanema,Simple lava (basic | U. crust),11.60,37.28,0.650215
4,1-CS-002-PR,Paranapanema,Simple lava (basic | massive interior),24.24,NaN,NaN


## Cumulative thickness of lava flows per geochem group
-----------------------------------------------------------------------------------------------------------------   
* This code runs throught the WellCad data and calculates the following:
    * Cumulative thickness of lavas per geochem group
    * Cumulative thickness of sedimentary beds per geochem group
    * Cumulative thickness of volcaniclastic beds per geochem group
-----------------------------------------------------------------------------------------------------------------   

In [25]:
## Cumulative thickness function definition v0.1

def cum_thickness (data, thick_dict, cumul_thick):
    'Calculates the cumulative thickness of lava flows per geochemical group'
    'Input: data - pandas df derived from wellcad int; thick_dict - empty dict to append unit:cul_thick pair; cumul_thick - empty list to append data to' 
    'Output: a dictionary contaning (unit:cumulative thickness) 
    'Output: a list with the same number of elements as input data containing cumulative flow thickness values. Used as a new row for input dataframe'
    
    # defining variables
    thick_counter = 0
    
    # Algorithm:
    for i in range(len(data)):
        if i == 0: # Setting up first iteration
            if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
            or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # If we're dealing with a new lava flow, grab thickness
                thick_counter += round(data['Flow thickness'].iloc[i])
                thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                cumul_thick.append(thick_counter)
            else:
                cumul_thick.append('0')
        else: # After first iteration
            if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before, keep going.
                if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
                or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # Look for facies with flow thickness data
                    thick_counter += round(data['Flow thickness'].iloc[i])
                    thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                    cumul_thick.append(thick_counter)
                else: # If no row with flow thickness data is being iterated
                    if re.search(r'\slava\s', data['Facies'].iloc[i]) != None: # If facies contain the word 'lava':
                        cumul_thick.append(thick_counter)
                    else:
                        cumul_thick.append('-')
            else: # If unit changes
                if data['Unit'].iloc[i] in thick_dict:  # If unit has already been added as a key to the thick_dict
                    thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                    thick_counter = round(thick_dict[data['Unit'].iloc[i]])
                    cumul_thick.append(thick_counter)
                else: # If a new unit is found
                    thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                    thick_counter = round(data['Flow thickness'].iloc[i])
                    thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                    cumul_thick.append(thick_counter)
                               
  

In [28]:
# Testing / Debugging
cs_pr_dict = {}
cs_pr_list = []

cum_thickness(data, cs_pr_dict, cs_pr_list)
print(cs_pr_dict, cs_pr_list)

{'Paranapanema': 37, 'Pitanga': 435, 'Esmeralda': 249, 'Vale do Sol': 135} ['0', 0, '-', 37, 37, 37, 3, 35, 35, 35, 58, 58, 58, 97, 97, 97, '-', 111, 117, 117, 117, 122, '-', 143, 143, 143, '-', 199, 199, 199, '-', 207, 272, 272, 272, '-', 306, 306, 306, 325, 325, 325, '-', 351, 351, 351, '-', 382, 382, 382, '-', 409, 409, 409, 435, 435, 435, 13, 26, 26, 26, '-', 75, 75, 75, '-', 87, 87, 87, '-', 158, 158, 158, 166, '-', 179, 179, 179, 195, 195, 195, 207, 207, 207, '-', 249, 249, 249, '-', 7, 14, 14, 14, 52, 52, 52, 63, 101, 101, 101, '-', 135, 135, 135]


-------------------------------------------------------------------------
# SEPARATION CELL
-------------------------------------------------------------------------

In [ ]:
## Cumulative thickness of lavas per geochem v0.2
# Output:
#   --> thick_dict (dict) containg the thickness of lava flow per geochem group
#   --> A cumulative thickness row is added to the data table.
## PROBLEMS --> Reallyyy verbose. Let's try to generalize this a bit by using regex expressions.


# Setting up variables:
thick_counter = 0
cumul_thick = []
thick_dict = {}

# Algorithm:
for i in range(len(data)):
    if i == 0: # Setting up first iteration
        if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
        or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # If we're dealing with a new lava flow, grab thickness
            thick_counter += round(data['Flow thickness'].iloc[i])
            thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
            cumul_thick.append(thick_counter)
        else:
            cumul_thick.append('0')
    else: # After first iteration
        if data['Unit'].iloc[i] == data['Unit'].iloc[i-1]: # If unit is the same as before, keep going.
            if data['Facies'].iloc[i] == 'Simple lava (basic | U. crust)' or data['Facies'].iloc[i] == 'Compound lava (basic)' \
            or data['Facies'].iloc[i] == 'Simple lava (basic | rubbly flow top)': # Look for facies with flow thickness data
                thick_counter += round(data['Flow thickness'].iloc[i])
                thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                cumul_thick.append(thick_counter)
            else: # If no row with flow thickness data is being iterated
                if re.search(r'\slava\s', data['Facies'].iloc[i]) != None: # If facies contain the word 'lava':
                    cumul_thick.append(thick_counter)
                else:
                    cumul_thick.append('-')
        else: # If unit changes
            if data['Unit'].iloc[i] in thick_dict:  # If unit has already been added as a key to the thick_dict
                thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                thick_counter = round(thick_dict[data['Unit'].iloc[i]])
                cumul_thick.append(thick_counter)
            else: # If a new unit is found
                thick_dict[data['Unit'].iloc[i-1]] = round(thick_counter)
                thick_counter = round(data['Flow thickness'].iloc[i])
                thick_dict[data['Unit'].iloc[i]] = round(thick_counter)
                cumul_thick.append(thick_counter)

# Checking
print(thick_dict)
data['Cumul thick'] = cumul_thick
data.head(84)

In [ ]:
# Regex debugging 
if re.search(r'\sU.\s', data['Facies'].iloc[0]) != None:
    print('Working!')
else:
    print('Not working :(')

In [ ]:
x = re.split(r'\s', data['Facies'].iloc[0])
print(re.search(r'U.', data['Facies'].iloc[0]))